In [18]:
import pandas as pd
import os

## 1. Download the data

In [8]:
data = 'viame_gfp_shlife'
output_dir = "/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/"
output_path = f"{output_dir}{data}.zip"
download_link = "https://viame.kitware.com/api/v1/dive_dataset/export?folderIds=[%2265a9a4b14af62c29c6612d7d%22,%2265a9a4f3cf5a99794eaca6ff%22,%2265a9a574cf5a99794eaca736%22,%2265a9a59b635fad85cd7d0b31%22,%2265a9a5c1cf5a99794eaca7b9%22,%2265a9a653a4c218785d4240b7%22,%2265a9a689a4c218785d42411e%22,%2265a9a6becf5a99794eaca8be%22,%2265a9a81578efbb739db7df70%22,%2265a9a847cf5a99794eaca996%22,%2265a9a87878efbb739db7e000%22,%2265a912dfb352f3bd2e5d608f%22,%2265a9a8a8b352f3bd2e5d83e6%22,%2265a9a8bc635fad85cd7d0e59%22,%2265a9a8ec4af62c29c66131f2%22,%2265a9a9084af62c29c661321f%22,%2265a9a93078efbb739db7e15b%22,%2265a9a955a4c218785d42449d%22,%2265a9abdba4c218785d42455d%22,%2265a9ac180c606d647bebd5fa%22,%2265a9ac3d78efbb739db7e2de%22,%2265a9ac4fa4c218785d42461a%22,%2265a9ac68a4c218785d424645%22,%2265a9acda635fad85cd7d11c5%22,%2265a9adbe0c606d647bebd8fa%22,%2265a9add978efbb739db7e5f8%22,%2265a9ad01635fad85cd7d1213%22,%2265a9ad11d17f7c8fa5b8e4d2%22,%2265a9ad14b352f3bd2e5d87f0%22,%2265a9ad2a0c606d647bebd7d8%22,%2265a9ad50cf5a99794eacaf69%22,%2265a9ad66d17f7c8fa5b8e578%22,%2265a9ad880c606d647bebd870%22,%2265a9ad8aa4c218785d424813%22,%2265a9aeaea4c218785d4249bc%22,%2265a9addd635fad85cd7d13b2%22,%2265a9adecd17f7c8fa5b8e685%22,%2265a9ae0bd17f7c8fa5b8e6d3%22,%2265a9ae290c606d647bebda0c%22,%2265b3f258a4c218785d45902a%22,%2265b3f262cf5a99794eae62f5%22,%2265b3f2870c606d647bed8d71%22,%2265b3f29ea4c218785d45907b%22,%2265b3f2a678efbb739db9a423%22,%2265b3f2aad17f7c8fa5ba975e%22,%2265b3f2af78efbb739db9a434%22,%2265b3f2b24af62c29c6642b0f%22,%2265b3f2cfd17f7c8fa5ba9799%22,%2265b3f2e5a4c218785d4590ea%22,%2265b3f2e7a4c218785d4590ef%22,%2265b3f2ef4af62c29c6642b9e%22,%2265b3f2f5d17f7c8fa5ba97e3%22,%2265b3f30278efbb739db9a4c7%22,%2265b3f31ab352f3bd2e5f3d6a%22,%2265b3f32c635fad85cd7ed029%22,%2265b3f32f4af62c29c6642c39%22,%2265b3f333a4c218785d4591a0%22]"

In [9]:
!wget -O {output_path} {download_link}
!unzip {output_path} -d {output_dir}

--2024-03-08 19:02:44--  https://viame.kitware.com/api/v1/dive_dataset/export?folderIds=[%2265a9a4b14af62c29c6612d7d%22,%2265a9a4f3cf5a99794eaca6ff%22,%2265a9a574cf5a99794eaca736%22,%2265a9a59b635fad85cd7d0b31%22,%2265a9a5c1cf5a99794eaca7b9%22,%2265a9a653a4c218785d4240b7%22,%2265a9a689a4c218785d42411e%22,%2265a9a6becf5a99794eaca8be%22,%2265a9a81578efbb739db7df70%22,%2265a9a847cf5a99794eaca996%22,%2265a9a87878efbb739db7e000%22,%2265a912dfb352f3bd2e5d608f%22,%2265a9a8a8b352f3bd2e5d83e6%22,%2265a9a8bc635fad85cd7d0e59%22,%2265a9a8ec4af62c29c66131f2%22,%2265a9a9084af62c29c661321f%22,%2265a9a93078efbb739db7e15b%22,%2265a9a955a4c218785d42449d%22,%2265a9abdba4c218785d42455d%22,%2265a9ac180c606d647bebd5fa%22,%2265a9ac3d78efbb739db7e2de%22,%2265a9ac4fa4c218785d42461a%22,%2265a9ac68a4c218785d424645%22,%2265a9acda635fad85cd7d11c5%22,%2265a9adbe0c606d647bebd8fa%22,%2265a9add978efbb739db7e5f8%22,%2265a9ad01635fad85cd7d1213%22,%2265a9ad11d17f7c8fa5b8e4d2%22,%2265a9ad14b352f3bd2e5d87f0%22,%2265a9ad2a0

## 2. Extract Frames
Some data is image sequence, so we already have the frames. Others are video sequences, so we need to extract them.


Therefore, have a ANNOTATIONS FIRST approach:
1. Extract the annotations
2. From the annotations select the frames you need

### 2.1 Convert annotations to sharktrack format

In [ ]:
SHARKTRACK_COLUMNS = ['filename', 'class', 'ymin', 'xmin', 'xmax', 'ymax', 'source', 'track_id', 'frame_id']

def viame2standard(csv_path, source, is_video, annotations_fps):
    assert annotations_fps in [1,10], "Only 1 or 10 fps are supported"
    # Load the CSV file
    df = pd.read_csv(csv_path, skiprows=lambda x: x in [1]) # skip row if metadata

    data = []

    for _, row in df.iterrows():
        # Build the Filename
        track_id = int(row["# 1: Detection or Track-id"])
        frame_id = int(row['3: Unique Frame Identifier'])

        if annotations_fps == 10 and frame_id % 10 != 0:
            # extract only 1fps annotations
            continue
    
        frame_id = int(frame_id / annotations_fps) # id starts at 0 and increments by 1 (if fps 1 doesn't change)

        if is_video:
            filename = f"frame{frame_id}.jpg"
        else:
            filename = row['2: Video or Image Identifier']
        img_path = os.path.join(output_dir, source, filename)
        assert os.path.exists(img_path), f"File {img_path} does not exist"

        species = row["10-11+: Repeated Species"]
        # TODO: filter out behaviour classification and clean only to shark/ray
        
        xmin = row['4-7: Img-bbox(TL_x']
        ymin = row['TL_y']
        xmax = row['BR_x']
        ymax = row['BR_y)']
        
        # Prepare the new row as a Series
        row = {
            'filename': f'{source}_{filename}',
            'class': species,
            'ymin': ymin,
            'xmin': xmin,
            'xmax': xmax,
            'ymax': ymax,
            'source': source,
            'track_id': track_id,
            'frame_id': frame_id,
        }

        data.append(row)

    converted_df = pd.DataFrame(data, columns=SHARKTRACK_COLUMNS)

    return converted_df

### 2.2 Move frames

In [ ]:
phase3_path = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3'

def process_image_sequence(image_sequence_path):
    source = os.path.basename(image_sequence_path)
    os.makedirs(os.path.join(phase3_path, source))

    # Convert annotations to sharktrack format
    #   annotations.csv(path)
    # Move frames there
    #   assert filename doesn't start with source and add it yourself
    #   rename frame to source_frame

In [ ]:
def process_video_annotations(video_path):
  # extract 1fps frames from video, name frame<id> where <id> starts at 0 and increments by 1
  # convert annotations to sharktrack format using same rule
  #   annotations.csv(path)
  # Move frames there
  #   assert filename doesn't start with source and add it yourself
  #   rename frame to source_frame

In [17]:
# If image: just move all the frames however they are named
# If video AND 1fps (sp), extract the frame at 1fps AND map it to annotation frame id
for video in os.listdir(output_dir):
  video_path = os.path.join(output_dir, video)
  if not os.path.isdir(video_path):
    continue

  for file in os.listdir(video_path):
    if file.endswith(".mp4"):
      print(f"{file} is video")
      break
    else:
      # Image Sequence!
      process_image_sequence(video_path)

easy2.transcoded.aligned.mp4 is video
medium2.mp4 is video
easy1.transcoded.aligned.mp4 is video
medium1.mp4 is video
difficult2.mp4 is video
difficult1.mp4 is video


## 3. Clean BBoxes

In [ ]:
# If image, just take the "2: Video or Image Identifier" column
# If 

## 4. Visualise Dataset and check everything is alright

## 5. Remove dumb FP